<a href="https://colab.research.google.com/github/KhosrojerdiA/Transformer-MT/blob/main/Transformer_Encoder_EXPLAINED!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import math
from torch import nn
import torch.nn.functional as F


In [ ]:
def scaled_dot_product(q, k, v, mask=None): #Attention (Q,K,V) = softmax (QKt/ sqrt(dk)V). single word is breaking down to QKV. #this is for one attention head
    d_k = q.size()[-1] #qkv - 30*8*200*64
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k) #30*8*200*200
    print(f"scaled.size() : {scaled.size()}")
    if mask is not None:
        print(f"-- ADDING MASK of shape {mask.size()} --")
        # Broadcasting add. So just the last N dimensions need to match #200*200 for every batch and every head: 30*80*200*200
        scaled += mask
    attention = F.softmax(scaled, dim=-1) #30*80*200*200
    values = torch.matmul(attention, v) #30*80*200*64 (64 dim embeddings)
    return values, attention

class MultiHeadAttention(nn.Module): #multi head

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model #512
        self.num_heads = num_heads #8
        self.head_dim = d_model // num_heads #64
        self.qkv_layer = nn.Linear(d_model , 3 * d_model) #512*1536
        self.linear_layer = nn.Linear(d_model, d_model) #512*512

    def forward(self, x, mask=None):
        batch_size, max_sequence_length, d_model = x.size() #30*200*512
        print(f"x.size(): {x.size()}")
        qkv = self.qkv_layer(x) #30*200*1536
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim) #30*200*8*192
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3)#30*8*200*192
        print(f"qkv.size(): {qkv.size()}")
        q, k, v = qkv.chunk(3, dim=-1) #each are #30*8*200*64
        print(f"q size: {q.size()}, k size: {k.size()}, v size: {v.size()}, ")
        values, attention = scaled_dot_product(q, k, v, mask) #attention #30*8*200*200 and #value #30*8*200*64
        print(f"values.size(): {values.size()}, attention.size:{ attention.size()} ")
        values = values.reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim) ##30*200*512
        print(f"values.size(): {values.size()}")
        out = self.linear_layer(values)
        print(f"out.size(): {out.size()}")
        return out


class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape #which dimension we want to perfomr layer norm. 512
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) #learnable paramater [512]
        self.beta =  nn.Parameter(torch.zeros(parameters_shape)) #learnable paramater [512]

    def forward(self, inputs): #30*200*512
        dims = [-(i + 1) for i in range(len(self.parameters_shape))] #[-1]
        mean = inputs.mean(dim=dims, keepdim=True) #30*200*1
        print(f"Mean ({mean.size()})")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True) #30*200*1
        std = (var + self.eps).sqrt() #30*200*1
        print(f"Standard Deviation  ({std.size()})")
        y = (inputs - mean) / std #30*200*512
        print(f"y: {y.size()}")
        out = self.gamma * y  + self.beta #30*200*512
        print(f"self.gamma: {self.gamma.size()}, self.beta: {self.beta.size()}")
        print(f"out: {out.size()}")
        return out


class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden) #512*2048
        self.linear2 = nn.Linear(hidden, d_model) #2048*512
        self.relu = nn.ReLU() #activation function. peacewise to capture information.
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):#30*200*512
        x = self.linear1(x) #30*200*2048
        print(f"x after first linear layer: {x.size()}")
        x = self.relu(x) #30*200*2048
        print(f"x after activation: {x.size()}")
        x = self.dropout(x)#30*200*2048
        print(f"x after dropout: {x.size()}")
        x = self.linear2(x)#30*200*512
        print(f"x after 2nd linear layer: {x.size()}")
        return x


class EncoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x): #sequence of executing is excatly like the digarams of Trnasformer
        residual_x = x #30*200*512
        print("------- ATTENTION 1 ------")
        x = self.attention(x, mask=None) #30*200*512
        print("------- DROPOUT 1 ------")
        x = self.dropout1(x) #30*200*512
        print("------- ADD AND LAYER NORMALIZATION 1 ------")
        x = self.norm1(x + residual_x) #30*200*512
        residual_x = x #30*200*512
        print("------- ATTENTION 2 ------")
        x = self.ffn(x) #30*200*512
        print("------- DROPOUT 2 ------")
        x = self.dropout2(x) #30*200*512
        print("------- ADD AND LAYER NORMALIZATION 2 ------")
        x = self.norm2(x + residual_x) #30*200*512
        return x

class Encoder(nn.Module): #inheritante from super class called Module. Cuda and access better
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers): #constructor.
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                     for _ in range(num_layers)]) #sequential to exe for every pass.

    def forward(self, x): #forward layer for propegation
        x = self.layers(x)
        return x



In [ ]:
d_model = 512 #size of the vectors (Dimensions for every word)
num_heads = 8 #for multi headed. 8 times in parallel (softmax and ATTR block)
drop_prob = 0.1 #drop out to learn from different path (regularization) 10% chance for neurons to be off.
batch_size = 30 #multiple examples together to pass to network for faster and stable training. Mini batch is better in more cases. 30 exampples to pro and gredinet calculation
max_sequence_length = 200 #max number of words we pass to network at a time
ffn_hidden = 2048 #feed foward network hidden layers.
num_layers = 5 #number of encoder we want to have in our model. this case, it is N=5.

encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

In [ ]:
x = torch.randn( (batch_size, max_sequence_length, d_model) ) # includes positional encoding
out = encoder(x)

------- ATTENTION 1 ------
x.size(): torch.Size([30, 200, 512])
qkv.size(): torch.Size([30, 200, 1536])
qkv.size(): torch.Size([30, 200, 8, 192])
qkv.size(): torch.Size([30, 8, 200, 192])
q size: torch.Size([30, 8, 200, 64]), k size: torch.Size([30, 8, 200, 64]), v size: torch.Size([30, 8, 200, 64]), 
scaled.size() : torch.Size([30, 8, 200, 200])
values.size(): torch.Size([30, 8, 200, 64]), attention.size:torch.Size([30, 8, 200, 200]) 
values.size(): torch.Size([30, 200, 512])
out.size(): torch.Size([30, 200, 512])
------- DROPOUT 1 ------
------- ADD AND LAYER NORMALIZATION 1 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 200, 512])
------- ATTENTION 2 ------
x after first linear layer: torch.Size([30, 200, 2048])
x after activation: torch.Size([30, 200, 2048])
x after dropout: torch.Size([30, 200, 2048])
x after 2nd linear layer: torch.